<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2025/blob/main/ExothermicTubularReactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

from sympy.abc import s
from sympy import exp, Symbol, simplify
import scipy as sp
import scipy.signal as sig

In [ ]:
N=5
q=100. #L/min
ca_in = 1. #mol/L
T_in = 350. #K
Vtotal = 100. #L
V = Vtotal/N  #L
rho = 1000. #g/L
C = 0.239 #J/(g K)
negHr = 5e4 #J/mol
ER = 8750. #K
k0 = 7.2e10 #1/min

UAtot = 5e4 #J/(min K)
UA = UAtot/N
Tc_ss = 300. #K
Ca_ss =  0.49991829 #mol/L
T_ss = 350.00552869 #K
tend = 10. #min

In [ ]:
def rhs(t, vec):
    Tc=305.
    ca, T = vec
    k=k0*np.exp(-ER/T)
    dca = q*(ca_in - ca)/V - k*ca
    dT = (rho*q*C*(T_in-T) - UA*(T-Tc) + negHr*V*k*ca)/(rho*V*C)
    return np.array([dca, dT])

In [ ]:
sp.optimize.root(lambda v: rhs(0., v), np.array([0.5, 350])).x

array([  0.6628935 , 337.96412992])

In [ ]:
res=sp.integrate.solve_ivp(rhs, (0, tend), (Ca_ss, T_ss), method='Radau', dense_output=True)

In [ ]:
tplot=np.linspace(0,tend,200)
cplot, Tplot = res.sol(tplot)
fig = make_subplots(rows=1, cols=2)
fig.add_scatter(x=tplot, y=cplot, row=1, col=1, name='Ca')
fig.add_scatter(x=tplot, y=Tplot, row=1, col=2, name='T')